# Creating an Ensemble Model

## Installs and imports

### Install all required libraries

In [1]:
# Uncomment line below to install all required libraries
# !pip3 install -r ../requirements.txt -q

### Import required libraries

In [2]:
import nltk
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection, naive_bayes, svm
import numpy as np
from sklearn.metrics import accuracy_score
import pickle
from tensorflow import keras

In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.initializers import constant
from keras.optimizers import Adam
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

## Load cleaned tweets dataset

In [4]:
df = pd.read_csv('./cleaned_tweets.csv')

In [5]:
np.random.seed(450)
df.head()

,sentiment,text,cleaned_tweet,Porter_Stem,Snowball_Stem
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",aww bummer shoulda got david carr third day,aww bummer shoulda got david carr third day,aww bummer shoulda got david carr third day
1,0,is upset that he can't update his Facebook by ...,upset cannot update facebook texting might cry...,upset can not updat facebook text might cri re...,upset can not updat facebook text might cri re...
2,0,@Kenichan I dived many times for the ball. Man...,dived many times ball managed save rest go bounds,dive mani time ball manag save rest go bound,dive mani time ball manag save rest go bound
3,0,my whole body feels itchy and like its on fire,whole body feels itchy like fire,whole bodi feel itchi like fire,whole bodi feel itchi like fire
4,0,"@nationwideclass no, it's not behaving at all....",behaving im mad cannot see,behav im mad can not see,behav im mad can not see


## Drop text

In [6]:
df = df[['sentiment', 'Snowball_Stem']]

In [7]:
df.head()

,sentiment,Snowball_Stem
0,0,aww bummer shoulda got david carr third day
1,0,upset can not updat facebook text might cri re...
2,0,dive mani time ball manag save rest go bound
3,0,whole bodi feel itchi like fire
4,0,behav im mad can not see


## Drop rows with NaN

In [8]:
df.isna().sum()

sentiment           0
Snowball_Stem    8046
dtype: int64

In [9]:
df = df.dropna()

In [10]:
df.isna().sum()

sentiment        0
Snowball_Stem    0
dtype: int64

# 5. Reduce dataframe size

In [11]:
df[df.sentiment != 0].shape

(795860, 2)

In [12]:
df[df.sentiment == 0].shape

(796094, 2)

In [13]:
df[df.sentiment != 0][:200000].shape

(200000, 2)

In [14]:
df[df.sentiment == 0][:200000].shape

(200000, 2)

In [15]:
reduced_df = pd.concat([df[df.sentiment != 0][740000:], df[df.sentiment == 0][740000:]])

In [16]:
reduced_df.shape

(111954, 2)

In [17]:
#df = reduced_df

In [18]:
X = df['Snowball_Stem']

In [19]:
Y = df['sentiment']

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [21]:
X_test.shape

(397989,)

## Loading the models

In [22]:
svm_path = './SVM_UnigramBigram_75.pickle'
nb_path = './NB_UnigramBigram_78.pickle'
lstm_token_path = './LSTM_tokenizer.pickle'
lstm_path = './LSTM_train_75_val_78_test_79_acc.h5'
vectorizer_path = './UnigramBigram_vectorizer.pickle'
RF_path = './RFC_UnigramBigram_72.pickle'
Blstm_path ='./B_LSTM_train_76_26_val_77_86_test_78_57_acc.h5'
blstm_token_path = './BLSTM_tokenizer.pickle'
# DT_path = 'DT_72.pickle'

In [23]:
svm = pickle.load(open(svm_path,'rb'))

In [24]:
nb = pickle.load(open(nb_path,'rb'))

In [25]:
tokenizer = pickle.load(open(lstm_token_path,'rb'))

In [26]:
lstm = keras.models.load_model(lstm_path)

In [27]:
token = pickle.load(open(blstm_token_path,'rb'))

In [28]:
vectorizer = pickle.load(open(vectorizer_path,'rb'))

In [29]:
RF = pickle.load(open(RF_path,'rb'))

In [30]:
Blstm = keras.models.load_model(Blstm_path)

# Encoding labels

In [31]:
Encoder = LabelEncoder()
y_test = Encoder.fit_transform(y_test)


## Running the models

In [32]:
xtest = vectorizer.transform(X_test)

### Naive bayes

In [33]:
NB3 = nb.predict(xtest)

In [34]:
NB_acc = accuracy_score(NB3, y_test)*100

In [35]:
NB_acc

77.87652422554392

### SVM

In [36]:
SVM_pred = svm.predict(xtest)

In [37]:
SVM_acc = accuracy_score(SVM_pred, y_test)*100

In [38]:
SVM_acc

77.36997756219392

### LSTM

In [39]:
max_seq_length = 20

In [40]:
test_tweet_seq = tokenizer.texts_to_sequences(X_test)

In [41]:
test_tweet_pad = pad_sequences(test_tweet_seq, maxlen=max_seq_length, padding="post", truncating="post")

In [42]:
test_tweet_pad[0]

array([   8,  582,  356,   28,  239,   80, 3031,    8,   32,   38,  279,
          0,    0,    0,    0,    0,    0,    0,    0,    0])

In [43]:
test_tweet_seq[0]

[8, 582, 356, 28, 239, 80, 3031, 8, 32, 38, 279]

In [44]:
lstm_pred = lstm.predict(test_tweet_pad)

In [45]:
LSTM_pred =[]
for i in lstm_pred:
    a = float(i)
    if a>=0.5:
        a = 1
    else:
        a = 0
    LSTM_pred.append(a)

In [46]:
LSTM_pred = np.array(LSTM_pred)

In [47]:
lstm_acc = accuracy_score(LSTM_pred, y_test)*100
lstm_acc

79.1798265781215

In [48]:
scores = lstm.evaluate(test_tweet_pad, y_test)
scores

12438/12438 [==============================] - 18s 1ms/step - loss: 0.4445 - accuracy: 0.7918


[0.44447553157806396, 0.7917982935905457]

## Random Forest

In [49]:
RF_pred = RF.predict(xtest)

In [50]:
RF_acc = accuracy_score(RF_pred, y_test)*100
RF_acc

73.10654314566483

## Bi Directional Lstm

In [51]:
seq_length = 20

In [52]:
ttweet_seq = token.texts_to_sequences(X_test)

In [53]:
ttweet_pad = pad_sequences(ttweet_seq, maxlen=seq_length, padding="post", truncating="post")

In [54]:
BLSTM_pred = Blstm.predict(ttweet_pad)

In [55]:
Blstm.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 20)            3587280   
_________________________________________________________________
bidirectional (Bidirectional (None, 64)                13568     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 3,600,913
Trainable params: 3,600,913
Non-trainable params: 0
_________________________________________________________________


In [56]:
scores = Blstm.evaluate(ttweet_pad, y_test)
scores

12438/12438 [==============================] - 28s 2ms/step - loss: 0.4533 - accuracy: 0.7864


[0.4532645642757416, 0.7863935828208923]

In [57]:
Blstm_pred =[]
for i in BLSTM_pred:
    a = float(i)
    if a>=0.5:
        a = 1
    else:
        a = 0
    Blstm_pred.append(a)

In [58]:
Blstm_pred = np.array(Blstm_pred)

In [59]:
Blstm_acc = accuracy_score(Blstm_pred, y_test)*100
Blstm_acc

78.63935937927933

## Putting predictions in dataframe

In [60]:
pdf = pd.DataFrame(y_test,columns=['Expected'])

In [61]:
pdf['SVM Predictions'] = SVM_pred
pdf['NB Predictions'] = NB3
pdf['LSTM Predictions'] = LSTM_pred
pdf['BLSTM Predictions'] = Blstm_pred
#pdf['Decision Tree Predictions'] = DT_pred

In [62]:
pdf.head()[1:3]

,Expected,SVM Predictions,NB Predictions,LSTM Predictions,BLSTM Predictions
1,1,1,1,1,1
2,0,0,1,1,1


In [63]:
lstm_acc

79.1798265781215

### Ensemble func

In [64]:
deno = (NB_acc + SVM_acc+ lstm_acc + Blstm_acc)
wNB = NB_acc/deno
wSVM = SVM_acc/deno
wLSTM = lstm_acc/deno
wBLSTM = Blstm_acc/deno
# wDT = DT_acc/deno
classWeights = [wSVM, wNB, wLSTM, wBLSTM]
classWeights

[0.24713656140170648,
 0.2487545817826636,
 0.25291761338783453,
 0.2511912434277955]

In [73]:
def ensemble_predict(a):
    pc = 0
    nc = 0
    for i in a:
        if i == 'Expected':
            continue
        if int(a[i]) ==1:
            pc+=1
        else:
            nc+=1
            
    p_pc = pc/(pc+nc)
    p_nc = 1-p_pc
    pscore =0
    nscore =0
    for w in classWeights:
        pscore+=w*pc
        nscore+=w*nc
    
    if pscore>=nscore:
        return 1
    else:
        return 0

In [74]:
l = len(pdf)
l

397989

In [75]:
ensPred = []
for i in range(l):
    ensPred.append(ensemble_predict(pdf[i:i+1]))

In [76]:
accuracy_score(ensPred, y_test)*100

79.50119224400675

In [69]:
svm_weighted_pred = pdf['SVM Predictions']*classWeights[0]
nb_weighted_pred = pdf['NB Predictions']*classWeights[1]
lstm_weighted_pred = pdf['LSTM Predictions']*classWeights[2]
blstm_weighted_pred = pdf['BLSTM Predictions']*classWeights[3]

In [70]:
weighted_pred_sum = svm_weighted_pred + nb_weighted_pred + lstm_weighted_pred + blstm_weighted_pred

In [71]:
pdf['Ensemble_predictions'] = list(map(lambda x: 1 if x>=0.5 else 0, weighted_pred_sum))

In [72]:
accuracy_score(pdf['Ensemble_predictions'],y_test)*100

79.50119224400675